In [1]:
# Load AIEEE Data

In [1]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [2]:
!nvidia-smi

Thu Mar 24 13:08:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   45C    P0    94W / 250W |   6169MiB / 16280MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

In [4]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [5]:
er_df = pd.read_csv("Models/Data/ERData/er_data_with_majority_label.csv")

Load Models

In [6]:
#  other models
AIEEE_Muril = "Models/AIEEEGenderData/MurilBERT/SavedModel/MurilBERT"
CBSE_Muril = "Models/CBSEData/MurilBERT/SavedModel/MurilBERT"
ER_Muril = "Models/ERData/MurilBERT/SavedModel/BERTMuril"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [9]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_Muril)

In [10]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_Muril)

In [11]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_Muril)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

Predict Gender

In [13]:
from transformers import pipeline
from sklearn.metrics import classification_report


In [14]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    report = classification_report(true, labels, digits = 4)
    print(classification_report(true, labels, digits = 4))
    return report

In [ ]:
report1 = print_results(aieee_model, tokenizer, er_df)

In [91]:
report2 = print_results(cbse_model, tokenizer, er_df)

              precision    recall  f1-score   support

         0.0     0.8646    0.9051    0.8844    175931
         1.0     0.9258    0.8932    0.9092    233355

    accuracy                         0.8983    409286
   macro avg     0.8952    0.8991    0.8968    409286
weighted avg     0.8995    0.8983    0.8985    409286

